# Conversión de archivo .data a archivo .xyz

Este notebook esta hecho de tal manera que solamente se necesita este notebook y la base de datos .data en la misma carpeta. El procesamiento, la creacion de carpetas y de archivos se maneja internamente en el notebook.

Debido a que el programa TorchMD-Net esta preparado para usar unidades de posición [ $\AA$], de fuerza,  [$\frac{eV}{\AA}$] y energía [eV]. Introducir otras unidades afecta de forma negativa la precisión de los resultados obtenidos. Transformaremos los datos del dataset original a las unidades esperadas.

Establecemos el nombre del archivo que contiene nuestros datos .data sin la extensión, esto para poder buscar el dataset y definir nombres de archivos.

In [ ]:
dataset_data= "input_bo222-200k-1173k"

Importamos los modulos necesarios

In [ ]:
import os
import shutil
import numpy as np
import h5py
import pandas as pd

Creamos las carpetas que necesitamos para el procesamiento del dataset, de existir las eliminamos y las volvemos a crear.

In [ ]:
#Carpetas del procesamiento del dataset
carpetas = ["originales", "procesados", "temp"]
carpeta_principal = "construccion-dataset"

def crear_carpeta(carpeta):
    ruta_carpeta = os.path.join(carpeta_principal, carpeta)
    os.makedirs(ruta_carpeta)
    print(f"La carpeta '{ruta_carpeta}/' ha sido creada.")

# Verificar y eliminar la carpeta principal si existe
if os.path.exists(carpeta_principal):
    shutil.rmtree(carpeta_principal)
    print(f"La carpeta '{carpeta_principal}/' existía y ha sido eliminada junto con su contenido.")

# Crear la carpeta principal de nuevo
os.makedirs(carpeta_principal)
print(f"La carpeta '{carpeta_principal}/' ha sido creada de nuevo.")

# Crear las carpetas dentro de la carpeta principal
for carpeta in carpetas:
    crear_carpeta(carpeta)

Devido a las unidades manejadas por el modelo, se recurre a convertir los datos al formato .xyz ya que este es un estándar.

Notemos que los datos de los cuales se obtendran los rasgos son: la posición de los átomos del arreglo y  su números átomicos. En cambio los targets son la energía y las fuerzas. 

Convertimos el archivo .data a .xyz frame a frame, notemos que convertirmos las unidades.

In [ ]:
# Factores de conversión
HARTREE_TO_EV=27.21138602
HA_BO_TO_EV_ANGS=51.422067137

# Abrir archivo
with open(dataset_data+".data", "r") as archivo:
    md_step=0
    # Iterar linea por linea
    for linea in archivo:
        # Convertir linea de texto a una lista python
        linea=linea.split()        
        if linea[0]=="begin":
            numero_de_atomos=0
            frame_atoms=""
        if linea[0]=="atom":
            numero_de_atomos=numero_de_atomos+1
            line_pos=[linea[1],linea[2],linea[3]]
            line_type=linea[4]
            line_force=[linea[7],linea[8],linea[9]]
            line_force = [float(elemento) * HA_BO_TO_EV_ANGS for elemento in line_force]
            line_pos = [float(val) for val in line_pos]
            line_force = [float(val) for val in line_force]
            line_data = f"{line_type:>2}{line_pos[0]:>20.10f}{line_pos[1]:>20.10f}{line_pos[2]:>20.10f}{line_force[0]:>20.10f}{line_force[1]:>20.10f}{line_force[2]:>20.10f}\n"

            frame_atoms=frame_atoms+line_data
        if linea[0]=='energy':
            frame_energy=float(linea[1])*HARTREE_TO_EV
            frame_energy = float(frame_energy)

        if linea[0]=='charge':
            continue
        if linea[0]=="end":

            frame=f"{numero_de_atomos}\n"
            frame = frame + f"MD_Step: {md_step} Total_energy = {frame_energy:.10f}\n"
            frame=frame+frame_atoms

            md_step=md_step+1

            with open(f"construccion-dataset/originales/{dataset_data}.xyz", "a") as file:
                file.write(frame)

## Conversión archivo .xyz a archivo .h5

El código ofrece posibilidades para nuevos datos, formatos .npy y .h5, en este notebook se escoje el formato .h5 debido a que se puede guardar el dataset completo en un solo archivo, cosa que no sucede con el formato .npy.

Para comprobar el correcto levantamiento de nuestros datos convertimos el archivo .xyz a un achivo .h5 con el cual podamos trabajar

Iteramos linea por linea el archivo .xyz y separamos los datos en archivos .dat, esto para no guardar los datos en ram y acceder a ellos en cada iteración lo cual hace muy pesado el procesamiento del dataset.

In [ ]:
numero_de_atomos=0
lista_num_atomos_en_muestras=[]
with open(f"construccion-dataset/originales/{dataset_data}.xyz", "r") as archivo:
    for linea in archivo:
        linea=linea.split()
        if len(linea)==1:
            numero_de_atomos=0
            frame_pos=""
            frame_type=""
            frame_force=""
            num_atomos_frame=linea[0]
        if len(linea)==5:
            frame_energy=float(linea[-1])
        if len(linea)==7:
            numero_de_atomos=numero_de_atomos+1
            line_pos=[linea[1],linea[2],linea[3]]
            line_type=[linea[0]]
            line_force=[linea[4],linea[5],linea[6]]           

            line_pos = " ".join(str(elemento) for elemento in line_pos)+ "\n"
            frame_pos=frame_pos+line_pos

            line_type = " ".join(str(elemento) for elemento in line_type)+ "\n"
            frame_type=frame_type+line_type

            line_force = " ".join(str(elemento) for elemento in line_force)+ "\n"
            frame_force=frame_force+line_force

        if int(numero_de_atomos)==int(num_atomos_frame):
            
            with open(f"construccion-dataset/temp/energy_{numero_de_atomos}.dat", "a") as file:
                file.write(str(frame_energy) + "\n")
                
            with open(f"construccion-dataset/temp/pos_{numero_de_atomos}.dat", "a") as file:
                file.write(frame_pos)
                
            with open(f"construccion-dataset/temp/type_{numero_de_atomos}.dat", "a") as file:
                file.write(frame_type)
                
            with open(f"construccion-dataset/temp/force_{numero_de_atomos}.dat", "a") as file:
                file.write(frame_force)

            lista_num_atomos_en_muestras.append(numero_de_atomos)

Generamos una lista con el numero de atomos en cada frame y un diccionario con el tamaño de frame diferentes y el numero de frames con ese tamaño. Esto nos servira para acomodar las dimensiones de nuestros numpy arrays.

In [ ]:
num_atomos_repeticiones = {}
num_atomos_unicos = list(set(lista_num_atomos_en_muestras))

# Iterar sobre la lista
for elemento in lista_num_atomos_en_muestras:
    # Verificar si el elemento ya está en el diccionario
    if elemento in num_atomos_repeticiones:
        # Si está presente, incrementar el contador
        num_atomos_repeticiones[elemento] += 1
    else:
        # Si no está presente, agregarlo al diccionario con contador 1
        num_atomos_repeticiones[elemento] = 1

Levantamos los datos de los archivos .dat, los cargamos en numpy arrays y los redimensionamos.

In [ ]:
def cargar_datos(archivo):
    if "type" in archivo:
        # Cargar los datos del archivo en un numpy array
        with open(archivo, 'r') as f:
            # Utilizar readlines() para leer todas las líneas del archivo
            contenido = f.readlines()
            # Eliminar los saltos de línea y convertir cada línea en una cadena
            datos = [linea.strip() for linea in contenido]
            # Convertir la lista de cadenas en un numpy array
            data = np.array(datos, dtype=object)
    else:
        # Cargar datos desde el archivo usando numpy.loadtxt
        data = np.loadtxt(archivo)
    return data
    
for num_atomos in num_atomos_unicos:
    # Lista de nombres de archivos
    archivos = [f"construccion-dataset/temp/energy_{num_atomos}.dat", f"construccion-dataset/temp/type_{num_atomos}.dat", f"construccion-dataset/temp/pos_{num_atomos}.dat", f"construccion-dataset/temp/force_{num_atomos}.dat"]
    
    # Crear una lista de arrays utilizando la función cargar_datos
    arrays = [cargar_datos(archivo) for archivo in archivos]

    # Asignar los arrays a variables específicas de forma dinámica
    for i, nombre_array in enumerate(['energy', 'types', 'pos', 'forces']):
        variable_name = f"{nombre_array}_{num_atomos}"
        if nombre_array == 'energy':
            globals()[variable_name]=np.reshape(arrays[i],(num_atomos_repeticiones[num_atomos]))
        if nombre_array == 'types':
            globals()[variable_name]=np.reshape(arrays[i],(num_atomos_repeticiones[num_atomos],num_atomos))
        if nombre_array == 'pos':
            globals()[variable_name]=np.reshape(arrays[i],(num_atomos_repeticiones[num_atomos],num_atomos,3))
        if nombre_array == 'forces':
            globals()[variable_name]=np.reshape(arrays[i],(num_atomos_repeticiones[num_atomos],num_atomos,3))

Guardamos los arrays en un archivo .h5

In [ ]:
archivo_h5=h5py.File(f"construccion-dataset/originales/{dataset_data}-original.h5", "w")

for num_atomos in num_atomos_unicos:
    group = archivo_h5.create_group(f"{num_atomos}_atoms")
    for nombre_array in ['energy', 'types', 'pos', 'forces']:
        array = f"{nombre_array}_{num_atomos}"
        group.create_dataset(nombre_array, data=globals()[array])    

archivo_h5.close()

El codigo es complicado debido a que esta preparado para funcionar para un dataset con arreglos con distinto número de átomos. El resto del notebook no esta optimizado para ese caso

# Contraste entre archivo .xyz y .h5 (originales)

Funciones para leer un archivo e imprimir un frame especifico

In [ ]:
def imprimir_muestra_h5(ruta_archivo_h5, grupo, muestra):
    import h5py
    archivo_h5 = h5py.File(ruta_archivo_h5, "r")
    grupo_actual = archivo_h5[grupo]
    num_atoms_frame = len(grupo_actual["types"][muestra])
    frame = f"{num_atoms_frame}\n"
    frame = frame + f"MD_Step: {muestra} Total_energy = {grupo_actual['energy'][muestra]:.10f}\n"
    for i in range(num_atoms_frame):
        if str(type(grupo_actual["types"][muestra][i]))=="<class 'bytes'>":
            line_type = grupo_actual["types"][muestra][i].decode()
        else:
            line_type = grupo_actual["types"][muestra][i]
        line_x = grupo_actual["pos"][muestra][i][0]
        line_y = grupo_actual["pos"][muestra][i][1]
        line_z = grupo_actual["pos"][muestra][i][2]
        if "forces" in grupo_actual.keys():
            line_fx = grupo_actual["forces"][muestra][i][0]
            line_fy = grupo_actual["forces"][muestra][i][1]
            line_fz = grupo_actual["forces"][muestra][i][2]
        else:
            line_fx = 0
            line_fy = 0
            line_fz = 0
        # Formatea la cadena con manejo de valores None
        linea = "{:>2}{:>20.10f}{:>20.10f}{:>20.10f}{:>20.10f}{:>20.10f}{:>20.10f}".format(
            line_type,
            line_x, 
            line_y, 
            line_z,
            line_fx, 
            line_fy, 
            line_fz
        )
        frame = frame + linea + "\n"
    print(frame)
    archivo_h5.close()

def imprimir_muestra_xyz(file_path, indice_muestra_deseada):
    with open(file_path, 'r') as archivo:
        conteo_muestras = 0

        for linea in archivo:
            if len(linea.split()) == 1:
                if conteo_muestras==indice_muestra_deseada:
                    frame=linea
                else:
                    pass
                numero_de_atomos=0
                num_atomos_frame=linea
            if len(linea.split()) == 5:
                if conteo_muestras==indice_muestra_deseada:
                    frame=frame+linea
                else:
                    pass               
            if len(linea.split()) == 7:
                if conteo_muestras==indice_muestra_deseada:
                    frame=frame+linea
                else:
                    pass
                numero_de_atomos=numero_de_atomos+1
            if int(numero_de_atomos) == int(num_atomos_frame):
                if conteo_muestras==indice_muestra_deseada:
                    print(frame)
                else:
                    pass
                conteo_muestras=conteo_muestras+1

Definimos el frame a corroborar

In [ ]:
muestra=10

Imprimimos el frame del archivo .xyz

In [ ]:
imprimir_muestra_xyz(f"construccion-dataset/originales/{dataset_data}.xyz", muestra)

Imprimimos el frame del archivo .h5

In [ ]:
imprimir_muestra_h5(f"construccion-dataset/originales/{dataset_data}-original.h5", "222_atoms", muestra)

# Transormación del dataset para TorchMD-Net

Hasta ahora transformamos los datos originales a un archivo .xyz para establecer un formato en común, después los convertimos al formato .h5 el cual es el formato que utiliza TorchMD-Net. Todo estas tranformaciónes tiene como fin dar certeza al correcto funcionamiento del procesamiento y no heredar errores a pasos futuros.

Es en este momento dónde vamos a transformar los datos a unos que puedan ser utilizados por el código.

Funciones a utilizar

In [ ]:
# Funcion para intercambiar el elemento atomico por su numero atomico
def periodic_table(input_elemento):
    tabla_periodica = {"H": 1, "O": 8, "Al": 13}

    if isinstance(input_elemento, str):
        # Si se proporciona el símbolo del elemento
        elemento = input_elemento.capitalize()
        if elemento in tabla_periodica:
            return tabla_periodica[elemento]
        else:
            return None
    elif isinstance(input_elemento, int):
        # Si se proporciona el número atómico
        for simbolo, numero_atomico in tabla_periodica.items():
            if numero_atomico == input_elemento:
                return simbolo
        # Si no se encuentra el número atómico en la tabla
        return None
    else:
        # Si el tipo de entrada no es válido
        return None

# Función para inercambiar el elemento atomico por su energía atómica
def atomics_energies(elemento_quimico):
    energias_atomicas = {
        1:-0.500272784191*27.21138602,
        8:-74.9555225243*27.21138602,
        13:-242.365764213*27.21138602,
        "H":-0.500272784191*27.21138602,
        "O":-74.9555225243*27.21138602,
        "Al":-242.365764213*27.21138602
    }
    elemento = int(elemento_quimico)
    if elemento in energias_atomicas:
        return energias_atomicas[elemento]
    else:
        return None

## Transformacion de los datos originales

TorchMD-Net requiere el número átomico y no el símbolo, además en entrenamientos anteriores se determinó que el target que genera los mejores resultados es la energia total del arreglo menos la energía de offset. Por lo que aplicamos estas correcciónes a los datos.

Iterar sobre los frames y transformar los datos segun lo requerido

In [ ]:
# Cargar archivo .h5
dataset_original=h5py.File(f"construccion-dataset/originales/{dataset_data}-original.h5", "r")
# Iterar sobre los grupos en el archivo
for grupo in dataset_original.keys():
    
    # Obtener el grupo actual
    grupo_actual = dataset_original[grupo]
    num_atoms= grupo[:-6]
    
    # Iterar sobre la energía de cada frame en cada grupo en el archivo .h5
    for i, frame_energy in enumerate(grupo_actual["energy"]):
        total_energy = frame_energy
        atom_energy = sum(map(atomics_energies, [periodic_table(elemento.decode()) for elemento in grupo_actual["types"][i]]))
        atoms_frame = len(grupo_actual["types"][i])
        new_frame_energy = total_energy-atom_energy
        with open(f"construccion-dataset/temp/energy_new_{num_atoms}.dat", "a") as file:
            file.write(str(new_frame_energy) + "\n")
    energy_array=cargar_datos(f"construccion-dataset/temp/energy_new_{num_atoms}.dat")
    globals()[f"energy_new_{num_atomos}"]=np.reshape(energy_array,(num_atomos_repeticiones[int(num_atomos)]))
            
    # Iterar sobre los simbolos atomicos de cada frame en cada grupo en el archivo .h5            
    for i, frame_types in enumerate(grupo_actual["types"]):
        new_frame_types = [periodic_table(elemento.decode()) for elemento in frame_types]
        with open(f"construccion-dataset/temp/types_new_{num_atoms}.dat", "a") as file:            
            file.write("\n".join(map(str, new_frame_types)) + "\n")
            
    types_array=cargar_datos(f"construccion-dataset/temp/types_new_{num_atoms}.dat")
    globals()[f"types_new_{num_atomos}"]=np.reshape(types_array,(num_atomos_repeticiones[int(num_atomos)],int(num_atomos)))

Crear nuevos archivos .h5 con los datos convertidos (con fuerzas y sin fuerzas)

In [ ]:
archivo_h5=h5py.File(f"construccion-dataset/procesados/{dataset_data}.h5", "w")
archivo_h5_f=h5py.File(f"construccion-dataset/procesados/{dataset_data}-f.h5", "w")

for num_atomos in num_atomos_unicos:
    
    group = archivo_h5.create_group(f"{num_atomos}_atoms")
    group.create_dataset("pos", data=dataset_original[f"{num_atomos}_atoms"]["pos"])
    group.create_dataset("types", data=globals()[f"types_new_{num_atomos}"][:].astype(np.int64))
    group.create_dataset("energy", data=globals()[f"energy_new_{num_atomos}"])
    
    group_f = archivo_h5_f.create_group(f"{num_atomos}_atoms")
    group_f.create_dataset("pos", data=np.array(dataset_original[f"{num_atomos}_atoms"]["pos"]))
    group_f.create_dataset("types", data=globals()[f"types_new_{num_atomos}"][:].astype(np.int64))
    group_f.create_dataset("energy", data=globals()[f"energy_new_{num_atomos}"])
    group_f.create_dataset("forces", data=dataset_original[f"{num_atomos}_atoms"]["forces"])

archivo_h5.close()
archivo_h5_f.close()

In [ ]:
dataset_original.close()

Guardar los arrays en archivos .npy
```python
for num_atomos in num_atomos_unicos:
    for nombre_array in ['energy', 'types', 'pos', 'forces']:
        array = f"{nombre_array}_{num_atomos}"
        np.save(f"construccion-dataset/originales/{array}.npy",globals()[array])
```

# Contraste archivo .h5 original contra archivo .h5 convertido

In [ ]:
muestra=10

In [ ]:
imprimir_muestra_h5(f"construccion-dataset/originales/{dataset_data}-original.h5", "222_atoms", muestra)

In [ ]:
imprimir_muestra_h5(f"construccion-dataset/procesados/{dataset_data}-f.h5", "222_atoms", muestra)

# Preparación del entrenamiento

Creamos las carpetas necesarias

In [ ]:
# Nombres de las carpetas que quieres crear
nombres_carpetas = ['input', 'output']

# Crear las carpetas si no existen y obtener las rutas relativas
rutas_carpetas = []
for carpeta in nombres_carpetas:
    os.makedirs(carpeta, exist_ok=True)
    rutas_carpetas.append(carpeta)

print("Carpetas creadas:")
for carpeta in rutas_carpetas:
    print(carpeta+"/")

Podemos heredar los indices de los conjuntos de datos (train, validation, test) de un archivo .csv o npz

In [ ]:
def archivo_csv():
    # Obtener la ruta absoluta del directorio actual
    directorio_actual = os.getcwd()
    
    # Buscar archivos en el directorio actual
    archivos_en_directorio = os.listdir(directorio_actual)
    for archivo in archivos_en_directorio:
        if os.path.isfile(archivo) and archivo.endswith('.csv'):
            return True
    
    # Si no se encuentra ningún archivo .csv
    return False

def archivo_npz():
    # Obtener la ruta absoluta del directorio actual
    directorio_actual = os.getcwd()
    
    # Buscar archivos en el directorio actual
    archivos_en_directorio = os.listdir(directorio_actual)
    for archivo in archivos_en_directorio:
        if os.path.isfile(archivo) and archivo.endswith('.npz'):
            return True
    
    # Si no se encuentra ningún archivo .npz
    return False

Si tenemos un achivo .csv lo convertimo a .npz con las caracteristicas necesarias

In [ ]:
if archivo_csv():
    
    # Obtener la ruta absoluta del directorio actual
    directorio_actual = os.getcwd()
    
    # Buscar archivos en el directorio actual
    archivos_en_directorio = os.listdir(directorio_actual)
    for archivo in archivos_en_directorio:
        if os.path.isfile(archivo) and archivo.endswith('.csv'):
            # Si se encuentra un archivo .csv, cargarlo y devolver el DataFrame
            archivo_csv_ = pd.read_csv(archivo)
            
    # Convertir la columna "ids" a tipo de datos int
    archivo_csv_["ids"] = archivo_csv_["ids"].astype(int)
    
    # Definir los tamaños de los conjuntos
    test_size = int(len(archivo_csv_) * 0.1)
    val_size = int(len(archivo_csv_) * 0.09)
    
    # Crear el diccionario para almacenar los conjuntos
    splits = {}
    splits['idx_test'] = archivo_csv_["ids"][:test_size].values
    splits['idx_val'] = archivo_csv_["ids"][test_size:test_size+val_size].values
    splits['idx_train'] = archivo_csv_["ids"][test_size+val_size:].values
    
    # Guardar los arrays en un archivo .npz
    np.savez("input/splits.npz", idx_train=splits['idx_train'], idx_val=splits['idx_val'], idx_test=splits['idx_test'])
    print("Se transformó el archivo .csv a .npz y se movió a input/")

Si tenemos un archivo .npz ya listo, simplemente lo copiamos a la carpeta correspondiente

In [ ]:
if archivo_npz():
    # Obtener la ruta absoluta del directorio actual
    directorio_actual = os.getcwd()
    
    # Buscar archivos en el directorio actual
    archivos_en_directorio = os.listdir(directorio_actual)
    
    # Verificar si el archivo especificado está en el directorio
    if "splits.npz" in archivos_en_directorio:

        # Nombre del archivo a mover
        nombre_archivo = "splits.npz"
        
        # Directorio actual
        directorio_origen = os.getcwd()
        directorio_destino = os.path.join(os.getcwd(), "input")
        
        # Ruta completa del archivo de origen y destino
        ruta_origen = os.path.join(directorio_origen, nombre_archivo)
        ruta_destino = os.path.join(directorio_destino, nombre_archivo)
        
        # Verificar si el archivo existe en el directorio actual
        if os.path.exists(ruta_origen):
            # Verificar si el directorio de destino existe, si no, crearlo
            if not os.path.exists(directorio_destino):
                os.makedirs(directorio_destino)
            
            # Mover el archivo
            shutil.move(ruta_origen, ruta_destino)
            print(f"El archivo '{nombre_archivo}' se ha movido correctamente a la carpeta 'input'.")
        else:
            print(f"No se encontró el archivo '{nombre_archivo}' en el directorio actual.")
    else:
        print("Revisa que tu archivo tenga las caracteristicas requeridas")


Si no tenemos ningun archivo, lo generamos nosotros. Debemos tener cuidado con el número de frames, en este caso se recupera el conteo del procesamiento del archivo .data

In [ ]:
if not archivo_csv() and not archivo_npz():
    numero_frames = md_step
    np.random.seed(42)
    numeros = np.arange(numero_frames)
    
    # Permutar aleatoriamente los números
    numeros_aleatorios = np.random.permutation(numeros)
    
    # Calcular las longitudes de los conjuntos de datos
    total_elementos = len(numeros_aleatorios)
    num_elementos_test = int(0.1 * total_elementos)
    num_elementos_train = int(0.81 * total_elementos)
    num_elementos_val = int(0.09 * total_elementos)
    
    # Dividir el array en tres conjuntos según las proporciones especificadas
    idx_test, idx_val, idx_train = np.split(numeros_aleatorios, [num_elementos_test, num_elementos_test + num_elementos_val])
    
    np.savez("input/splits.npz", idx_test=idx_test, idx_train=idx_train, idx_val=idx_val)

    print("Como no se encontraron archivos .npz o .csv se generaron los ids de cada conjunto de datos")

Copiamos el dataset procesado el con el cual entrenaremos

In [ ]:
# Directorio actual
directorio_actual = os.getcwd()

# Rutas de origen y destino (relativas al directorio actual)
ruta_origen_relativa = f"construccion-dataset/procesados/{dataset_data}-f.h5"
ruta_destino_relativa = "input"

# Rutas completas de origen y destino
ruta_origen = os.path.join(directorio_actual, ruta_origen_relativa)
ruta_destino = os.path.join(directorio_actual, ruta_destino_relativa)

# Copiar el archivo
shutil.copy(ruta_origen, ruta_destino)

# Verificar que el archivo se ha copiado correctamente
archivo_copiado = os.path.join(ruta_destino, os.path.basename(ruta_origen))

if os.path.exists(archivo_copiado):
    print(f"El archivo se ha copiado correctamente a {archivo_copiado}")
else:
    print("Hubo un problema al copiar el archivo.")

Generamos los archivos .xyz correspondientes a cada subconjunto de datos para poder saber que datos se utilizaron en cada subconjunto.

In [ ]:
splits = np.load("input/splits.npz")
idx_train = np.sort(splits['idx_train'])
idx_val = np.sort(splits['idx_val'])
idx_test = np.sort(splits['idx_test'])

with open(f"construccion-dataset/originales/{dataset_data}.xyz", 'r') as archivo:
    
    conteo_muestras = 0
    
    for linea in archivo:
        if len(linea.split()) == 1:
            frame=linea
            numero_de_atomos=0
            num_atomos_frame=linea
        if len(linea.split()) == 5:
            frame=frame+linea             
        if len(linea.split()) == 7:
            frame=frame+linea
            numero_de_atomos=numero_de_atomos+1
        if int(numero_de_atomos) == int(num_atomos_frame):
            frame=frame+linea
            indice= conteo_muestras
            if indice in idx_train:
                with open(f"construccion-dataset/originales/{dataset_data}-train.xyz", 'a') as output_file:
                    output_file.write(frame)
            if indice in idx_val:
                with open(f"construccion-dataset/originales/{dataset_data}-val.xyz", 'a') as output_file:
                    output_file.write(frame)
            if indice in idx_test:
                with open(f"construccion-dataset/originales/{dataset_data}-test.xyz", 'a') as output_file:
                    output_file.write(frame)
                
            conteo_muestras += 1

Definimos la configuración del entrenamiento y la guardamos en un archivo .yaml

In [ ]:
configuracion = f"""activation: silu
aggr: add
atom_filter: -1
attn_activation: silu
batch_size: 32
coord_files: null
cutoff_lower: 0.0
cutoff_upper: 6.35
dataset: HDF5
dataset_arg: null
dataset_root: input/{dataset_data}-f.h5
derivative: True
distance_influence: both
early_stopping_patience: 500
ema_alpha_neg_dy: 1.0
ema_alpha_y: 1.0
embed_files: null
embedding_dimension: 128
energy_files: null
y_weight: 0.0001
force_files: null
neg_dy_weight: 1.0
inference_batch_size: 4
load_model: null
log_dir: logs/
lr: 0.0007
lr_factor: 0.5
lr_min: 1.0e-07
lr_patience: 5
lr_warmup_steps: 10000
max_num_neighbors: 140
max_z: 100
model: equivariant-transformer
neighbor_embedding: true
ngpus: -1
num_epochs: 1000
num_heads: 8
num_layers: 6
num_nodes: 1
num_rbf: 32
num_workers: 6
output_model: Scalar
precision: 32
rbf_type: expnorm
redirect: false
reduce_op: add
save_interval: 10
seed: 42
splits: input/splits.npz
standardize: false
test_interval: 10
test_size: 0.1
train_size: 0.81
trainable_rbf: false
val_size: 0.09
weight_decay: 0.0
"""

# Ruta del archivo YAML de salida
ruta_salida = f"input/{dataset_data}.yaml"

# Guardar el texto como archivo YAML sin módulos adicionales
with open(ruta_salida, 'w') as archivo:
    archivo.write(configuracion)

print(f"El archivo YAML se ha guardado correctamente en: {ruta_salida}")

Definimos el script de entrenamiento que sera ejecutado por nohup, es importante tomar en cuenta que el comando CUDA_VISIBLE_DEVICES Hace referencia a las gpu's que se utilizaran.

In [ ]:
comando_principal=f"CUDA_VISIBLE_DEVICES=0,1,2,3 torchmd-train --conf {ruta_salida} --log-dir output/"

texto_script=f"""#!/bin/bash

# Archivo para guardar la información
output_file="output/informacion_entrenamiento.txt"

# Guardar el comando de ejecución y la hora de inicio
echo "Comando de ejecución: {comando_principal}" > "$output_file"
start_time=$(date +"%Y-%m-%d %H:%M:%S")
echo "Inicio del entrenamiento: $start_time" >> "$output_file"

# Ejecutar el comando de entrenamiento
{comando_principal}

# Guardar la hora de finalización
end_time=$(date +"%Y-%m-%d %H:%M:%S")
echo "Fin del entrenamiento: $end_time" >> "$output_file"

# Calcular la duración del entrenamiento
start_seconds=$(date -d "$start_time" +%s)
end_seconds=$(date -d "$end_time" +%s)
duration_seconds=$((end_seconds - start_seconds))

# Guardar la duración en el archivo
echo "Duración del entrenamiento: $((duration_seconds / 60)) minutos y $((duration_seconds % 60)) segundos" >> "$output_file"

echo "Información del entrenamiento guardada en $output_file"
"""

# Ruta del script de salida
ruta_salida = "iniciar-entrenamiento.sh"

# Guardar el texto como archivo YAML sin módulos adicionales
with open(ruta_salida, 'w') as archivo:
    archivo.write(texto_script)

print(f"El script se ha guardado correctamente en: {ruta_salida}")

# Ejecución del entrenamiento

In [ ]:
!pwd

Recuerda dar permiso de ejecución al script con el comando en terminal

```bash
chmod +x iniciar-entrenamiento.sh
```

Ejecución con nohup

Para que el script se siga ejecutando independientemente del estado de la terminal donde se inicio se ejecuta el siguiente comando

```bash
nohup ./iniciar-entrenamiento.sh > output/salida.log 2>&1 &
```